# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [1]:
#load python packages
import os
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [4]:
resort = pd.read_csv('/Users/HOME/Desktop/Springboard/GuidedCapstone_step_2/data/step3_output.csv', index_col=0)
resort.reset_index(drop = True, inplace = True)

In [5]:
resort.head()

,Name,state,summit_elev,vertical_drop,fastQuads,quad,triple,double,surface,total_chairs,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Hilltop Ski Area,Alaska,2090,294,0,0,1,0,2,3,...,30.0,30.000000,150.000000,36.0,69.0,30.000000,34.00000,152.0,30.0,0
1,Sunrise Park Resort,Arizona,11100,1800,1,2,3,1,0,7,...,800.0,80.000000,115.000000,49.0,250.0,74.000000,78.00000,104.0,80.0,1
2,Yosemite Ski & Snowboard Area,California,7800,600,0,0,1,3,1,5,...,88.0,174.873239,110.000000,84.0,300.0,47.000000,47.00000,107.0,0.0,1
3,Donner Ski Ranch,California,8012,750,0,0,1,5,2,8,...,505.0,60.000000,163.000000,82.0,400.0,75.000000,75.00000,170.0,0.0,1
4,June Mountain,California,10090,2590,2,0,0,4,1,7,...,1500.0,174.873239,115.103943,58.0,250.0,57.916957,64.16681,128.0,0.0,1


## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [6]:
resort = pd.concat([resort.drop('state', axis=1), pd.get_dummies(resort['state'])], axis=1)
resort.T

,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
Name,Hilltop Ski Area,Sunrise Park Resort,Yosemite Ski & Snowboard Area,Donner Ski Ranch,June Mountain,Mt. Shasta Ski Park,Mountain High,Ski China Peak,Soda Springs,Tahoe Donner,...,Mount La Crosse,Nordic Mountain,Sunburst,Trollhaugen,Whitecap Mountain,Hogadon Basin,Sleeping Giant Ski Resort,Snow King Resort,Snowy Range Ski & Recreation Area,White Pine Ski Area
summit_elev,2090,11100,7800,8012,10090,6890,8200,8709,7352,7350,...,1110,1137,1100,1200,1750,8000,7428,7808,9663,9500
vertical_drop,294,1800,600,750,2590,1435,1600,1679,652,600,...,516,265,214,260,400,640,810,1571,990,1100
fastQuads,0,1,0,0,2,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
quad,0,2,0,0,0,0,2,1,0,1,...,0,0,0,2,1,0,0,1,0,0
triple,1,3,1,1,0,3,2,4,1,1,...,0,1,0,0,0,0,1,1,1,2
double,0,1,3,5,4,0,5,2,1,0,...,3,1,3,1,4,1,1,1,3,0
surface,2,0,1,2,1,1,3,4,2,3,...,1,5,6,5,0,1,1,0,1,0
total_chairs,3,7,5,8,7,4,14,11,4,5,...,4,7,9,8,5,2,3,3,5,2
Runs,13,65,10,52,41,32,59,45,18,14,...,19,18,13,24,43,28,48,32,33,25


## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [7]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = resort.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = resort.AdultWeekend 

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [8]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [9]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [10]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_tes
y_pred = model.predict(X_test)

## Review Model Outcomes — Iterate over additional models as needed

In [0]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model

In [11]:
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
#mean_absolute_error - Best possible score is 1.0, lower values are worse
evs = explained_variance_score(y_test,y_pred)
print("explained_variance_score for model1: ",evs)
mbe = mean_absolute_error(y_test,y_pred)
print("mean_absolute_error for model1: ",mbe)

explained_variance_score for model1:  -5.449837312827819e+22
mean_absolute_error for model1:  846999803652.5388


**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [12]:
print("model1 intercept: ",lm.intercept_)

model1 intercept:  -213017914633.45233


**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [13]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])


,Coefficient
summit_elev,2.017959e+00
vertical_drop,5.732599e-01
fastQuads,8.882909e+11
quad,1.117027e+12
triple,1.847301e+12
double,2.164603e+12
surface,2.236777e+12
total_chairs,3.915428e+12
Runs,2.126168e+00
TerrainParks,3.273926e-01


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [14]:
resort.drop(resort.loc[:,'Alaska':'Wyoming'],axis = 1,inplace = True)

In [15]:
X = resort.drop(['Name','AdultWeekend'], axis=1)
Y = resort.AdultWeekend
scaler = preprocessing.StandardScaler().fit(X)
X_scaled=scaler.transform(X) 
y = Y.ravel()

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

In [16]:
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)
y_pred = lm.predict(X_test)

In [17]:
evs = explained_variance_score(y_test,y_pred)
print("explained_variance_score for model2: ",evs)
mbe = mean_absolute_error(y_test,y_pred)
print("mean_absolute_error for model2: ",mbe)
print("model2 intercept: ",lm.intercept_)

explained_variance_score for model2:  0.553888701572987
mean_absolute_error for model2:  6.87494180049977
model2 intercept:  56.21849691437927


In [18]:
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])

,Coefficient
summit_elev,3.410061
vertical_drop,1.281556
fastQuads,0.164068
quad,0.340800
triple,1.371843
double,0.503317
surface,1.140739
total_chairs,0.408265
Runs,1.689551
TerrainParks,0.062112


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [19]:
X = resort.drop(['Name','AdultWeekend','summit_elev','vertical_drop'], axis=1)
Y = resort.AdultWeekend
scaler = preprocessing.StandardScaler().fit(X)
X_scaled=scaler.transform(X) 
y = Y.ravel()

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

In [20]:
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)
y_pred = lm.predict(X_test)
evs = explained_variance_score(y_test,y_pred)
print("explained_variance_score for model3: ",evs)
mbe = mean_absolute_error(y_test,y_pred)
print("mean_absolute_error for model3: ",mbe)
print("model2 intercept: ",lm.intercept_)

explained_variance_score for model3:  0.5525678408596658
mean_absolute_error for model3:  7.086889803440092
model2 intercept:  56.15778063689167


In [21]:
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])

,Coefficient
fastQuads,0.390608
quad,0.101619
triple,1.351959
double,0.680799
surface,0.971343
total_chairs,0.576934
Runs,1.796288
TerrainParks,0.200886
LongestRun_mi,0.070363
SkiableTerrain_ac,1.065975


In [22]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df.head(25)

,Actual,Predicted
0,47.00000,54.133140
1,49.00000,52.420973
2,68.00000,46.802136
3,59.00000,65.197778
4,59.00000,53.814244
5,71.00000,64.899029
6,58.00000,48.778511
7,55.00000,39.438953
8,54.00000,44.499369
9,65.00000,67.033090


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0. |  |-|
| Model 2. | 0.| |'state'|
| Model 3. | 0. | |'state','summit_elev','base_elev'|

Model Selection:

In [0]:
Model3 is good because of Mean Absolute Error 